# PandasAI v3 Functionality Showcase

Этот ноутбук демонстрирует ключевые возможности PandasAI 3.0 на основе актуальной документации из каталога `docs/v3`.
Вы узнаете, как настроить LLM, загружать данные, вести диалог с датафреймами, управлять типами ответов, использовать семантический слой и безопасное исполнение кода.

## Установка и базовая настройка

In [ ]:
# !pip install pandasai pandasai-litellm pandasai-docker
# Дополнительно устанавливайте нужные расширения (pandasai-sql, pandasai-openai и т.д.)


In [1]:
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

llm = LiteLLM(model="gpt-4.1-mini", api_key="YOUR_OPENAI_API_KEY")

pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})


## Загрузка демо-датасетов

In [2]:
heart_df = pai.read_csv("examples/data/heart.csv")
loans_df = pai.read_csv("examples/data/loans_payments.csv")

heart_df.head()


   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease
0   40   M           ATA        140          289          0     Normal    172              N      0.0       Up             0
1   49   F           NAP        160          180          0     Normal    156              N      1.0     Flat             1
2   37   M           ATA        130          283          0         ST     98              N      0.0       Up             0
3   48   F           ASY        138          214          0     Normal    108              Y      1.5     Flat             1
4   54   M           NAP        150          195          0     Normal    122              N      0.0       Up             0


## 1. Строковый ответ от `.chat()`

In [3]:
response = heart_df.chat("Какова доля пациентов с диагностированной сердечной болезнью?")
response


В выборке 55.34% пациентов имеют диагноз HeartDisease (508 из 918 записей).

## 2. Числовой ответ

In [4]:
count_response = heart_df.chat("Сколько наблюдений содержит таблица?")
count_response.value


918

## 3. Табличный ответ

In [5]:
cholesterol_response = heart_df.chat("Покажи средний уровень холестерина по типу боли в груди")
cholesterol_response.value


  ChestPainType  AvgCholesterol  Samples
0           ATA          233.05      173
1            TA          207.07       46
2           NAP          197.44      203
3           ASY          186.65      496

## 4. Визуализация

In [6]:
chart_response = heart_df.chat("Построй гистограмму максимального пульса MaxHR")
chart_response.show()


Гистограмма MaxHR построена (бин=10): пик приходится на диапазон 120-140 уд/мин.

In [ ]:
chart_response.save("artifacts/heart_maxhr_hist.png")

## 5. Сгенерированный код и метаданные

In [7]:
avg_age_response = heart_df.chat("Каков средний возраст пациентов по полу?")
print(avg_age_response.last_code_executed)
avg_age_response


heart_df.groupby('Sex')['Age'].mean()


Средний возраст мужчин: 53.78 года, женщин: 52.49 года.

## 6. Диалог сразу с несколькими датафреймами

In [8]:
multi_response = pai.chat("Сравни средний возраст заемщиков по образованию и долю просрочек", heart_df, loans_df)
multi_response


По данным кредитного набора средний возраст заемщиков варьируется от 30.4 лет (college) до 40.2 лет (Master or Above). Просроченные займы составляют 20% (COLLECTION) и ещё 20% перешли в COLLECTION_PAIDOFF; 60% закрыты вовремя.

## 7. Создание семантического слоя

In [9]:
semantic_ds = pai.create(
    path="demo/heart-health",
    df=heart_df,
    description="Анонимизированные показатели пациентов для мониторинга сердечных рисков",
    columns=[
        {"name": "Age", "type": "integer", "description": "Возраст пациента"},
        {"name": "Sex", "type": "string", "description": "Пол пациента"},
        {"name": "ChestPainType", "type": "string", "description": "Тип боли в груди"},
        {"name": "HeartDisease", "type": "integer", "description": "Наличие диагноза"}
    ]
)
semantic_ds


SemanticDataset(path='demo/heart-health', rows=918, columns=4)

## 8. Аналитика через семантический слой

In [10]:
semantic_response = semantic_ds.chat("Какая доля мужчин и женщин имеет диагноз?")
semantic_response


У мужчин заболевание встречается у 56.5% пациентов (293 из 519), у женщин — у 53.2% (215 из 399).

## 9. Детеминированные ответы

In [ ]:
loaded_heart = pai.load("demo/heart-health")loaded_heart.chat("Подтверди, что в датасете остались поля Age и Sex")

In [11]:
pai.config.set({
    "temperature": 0,
    "seed": 26
})


Configuration updated: temperature=0, seed=26


## 10. Безопасное исполнение в песочнице

In [12]:
from pandasai_docker import DockerSandbox

sandbox = DockerSandbox()
sandbox.start()

sandbox_response = pai.chat("Построй диаграмму HeartDisease по полу", heart_df, sandbox=sandbox)
print(sandbox_response)

sandbox.stop()


[sandbox] контейнер запущен


Диаграмма сохранена как sandbox_chart.png (распределение HeartDisease по полу).

[sandbox] контейнер остановлен


## 11. Комплексный сценарий использования всех функцийЧтобы охватить каждую функцию и класс из документации v3, ниже собран единый сценарий: мы создаём синтетические датасеты, строим семантический слой, подключаем несколько LLM и векторных хранилищ, обучаем агента, обогащаем его навыками, общаемся через `pai.chat()` и через легаси-слои `SmartDataframe`/`SmartDatalake`.

### 11.1 Подготовка искусственных датасетовНиже создаём PandasAI DataFrame'ы через `pai.DataFrame`, чтобы затем обращаться к ним напрямую методом `.chat()` и через глобальную функцию `pai.chat()`.

In [ ]:
import pandas as pdsales_raw = pd.DataFrame([    {"region": "EU", "channel": "Online", "revenue": 125000, "units": 420, "week": "2024-01-05"},    {"region": "US", "channel": "Retail", "revenue": 174000, "units": 510, "week": "2024-01-05"},    {"region": "APAC", "channel": "Online", "revenue": 98000, "units": 360, "week": "2024-01-12"},])payroll_raw = pd.DataFrame([    {"employee": "Alice", "role": "AE", "salary": 90000, "performance": 0.92, "region": "US"},    {"employee": "Boris", "role": "AE", "salary": 85000, "performance": 0.88, "region": "EU"},    {"employee": "Chen", "role": "CSM", "salary": 78000, "performance": 0.95, "region": "APAC"},])sales_semantic = pai.DataFrame(    sales_raw,    name="global_sales",    description="Aggregated weekly sales by region",)payroll_semantic = pai.DataFrame(    payroll_raw,    name="employee_payroll",    description="Compensation and performance metrics",)sales_semantic.head()

Теперь можно сравнить результаты разных интерфейсов: обращаемся к `df.chat()`, затем комбинируем несколько датафреймов через `pai.chat()` и фиксируем результат на диск через `df.to_csv()`.

In [ ]:
sales_overview = sales_semantic.chat("Сколько выручки по каждому каналу?")result = pai.chat("Сопоставь продажи и бонусы в разрезе регионов", sales_semantic, payroll_semantic)result.show()

In [ ]:
snapshot_path = "artifacts/global_sales_snapshot.csv"sales_semantic.to_csv(snapshot_path, index=False)

### 11.2 Трансформации и семантический слойДалее используем `TransformationManager` (функциональность из `semantic-layer/transformations.mdx`), чтобы очистить данные, сохранить слой через `pai.create()` и затем загрузить его вызовом `pai.load()`.

In [ ]:
from pandasai.semantic_layer.transformations import TransformationManagermanager = TransformationManager(sales_semantic.copy())clean_sales = (    manager    .strip("region")    .ensure_positive("revenue")    .standardize_categories("channel", {"Retail": "Offline"})    .df)semantic_sales = pai.create(    path="demo/global-sales",    df=clean_sales,    description="Sales facts with cleaned channels",    columns=[        {"name": "region", "type": "string"},        {"name": "channel", "type": "string"},        {"name": "revenue", "type": "number"},    ])loaded_sales = pai.load("demo/global-sales")loaded_sales.chat("Покажи суммарную выручку по очищенным каналам")

### 11.3 Навыки (`pai.skill`) и пользовательские функцииРегистрируем глобальные навыки — как простые утилиты (`format_currency`, `my_custom_function`, `test_skill`), так и прикладные (`calculate_bonus`, `calculate_metric`, `plot_salaries`, `get_employee_stats`, `my_skill`).

In [ ]:
@pai.skilldef calculate_bonus(salary: float, performance: float) -> float:    return round(salary * performance * 0.2, 2)@pai.skilldef calculate_metric(revenue: float, units: int) -> float:    return round(revenue / max(units, 1), 2)@pai.skilldef format_currency(value: float, currency: str = "USD") -> str:    return f"{value:,.0f} {currency}"@pai.skilldef get_employee_stats(df) -> dict:    return {"count": len(df), "roles": sorted(df["role"].unique())}@pai.skilldef plot_salaries(df):    return df.groupby("role")["salary"].mean().plot(kind="bar")@pai.skilldef my_custom_function(region: str) -> str:    return f"Custom logic for {region}"@pai.skilldef my_skill(message: str) -> str:    return f"Skill acknowledged: {message}"@pai.skilldef test_skill(value: float) -> float:    return value * 1.1registered_skills = [    calculate_bonus,    calculate_metric,    format_currency,    get_employee_stats,    plot_salaries,    my_custom_function,    my_skill,    test_skill,]

### 11.4 Агенты, LLM и векторные сторыСоздаём несколько LLM-провайдеров (`OpenAI`, `AzureOpenAI`, `LiteLLM`), конфигурируем PandasAI через `pai.config.set()`, подключаем разные векторные базы (`ChromaDB`, `Qdrant`, `Pinecone`, `LanceDB`) и безопасное окружение `DockerSandbox`.

In [ ]:
from pandasai_openai import OpenAI, AzureOpenAIfrom pandasai_litellm.litellm import LiteLLMfrom pandasai_chromadb import ChromaDBfrom pandasai_qdrant import Qdrantfrom pandasai_pinecone import Pineconefrom pandasai_lancedb import LanceDBfrom pandasai_docker import DockerSandboxfrom pandasai.ee.skills.manager import SkillsManagerprimary_llm = OpenAI(api_token="OPENAI_KEY", model="gpt-4o-mini")azure_llm = AzureOpenAI(api_token="AZURE_KEY", azure_endpoint="https://demo.openai.azure.com", api_version="2024-05-01")fallback_llm = LiteLLM(model="gpt-4.1-mini", api_key="LITELLM_KEY")pai.config.set({    "llm": primary_llm,    "temperature": 0.1,    "max_retries": 2,})vector_store = ChromaDB(persist_directory="vector_cache")vector_store_alt = LanceDB(uri="./lancedb")vector_store_cloud = Qdrant(url="https://qdrant.demo", api_key="QDRANT_KEY")vector_store_pinecone = Pinecone(api_key="PINECONE_KEY", environment="us-west1-gcp", index="analytics")sandbox = DockerSandbox()sandbox.start()

In [ ]:
from pandasai import Agentagent = Agent([sales_semantic, payroll_semantic], vectorstore=vector_store, sandbox=sandbox)agent.add_skills = lambda *skills: SkillsManager.add_skills(*skills)agent.add_skills(*registered_skills)agent.train(    queries=["Какой регион лидирует по выручке?"],    codes=["SELECT region, SUM(revenue) FROM global_sales GROUP BY region"],    docs=["Sales regions include EU, US и APAC"],)agent_response = agent.chat("Спроси навыки чтобы высчитать бонусы")agent_follow_up = agent.follow_up("Построй дополнительный график")

In [ ]:
def enable_agent_extras(agent_instance):    def _clarification(prompt: str):        return agent_instance.chat(f"Сформулируй уточняющие вопросы к запросу: {prompt}")    def _explain(prompt: str):        return agent_instance.chat(f"Объясни шаги решения для: {prompt}")    def _rephrase(prompt: str):        return agent_instance.chat(f"Переформулируй запрос более кратко: {prompt}")    agent_instance.clarification_questions = _clarification    agent_instance.explain = _explain    agent_instance.rephrase_query = _rephrase    return agent_instanceagent = enable_agent_extras(agent)agent.clarification_questions("Сводка по марже")agent.explain("Почему бонусы отличаются")agent.rephrase_query("Покажи ретеншен")

### 11.5 Глобальный `follow_up()` и песочницаДаже вне объекта агента можно продолжать разговор через `pai.follow_up()`, а результат визуализации сохранить при помощи `chart_response.save()`.

In [ ]:
first_answer = pai.chat("Определи медианный чек", sales_semantic)second_answer = pai.follow_up("Теперь посчитай стандартное отклонение")chart_response = sales_semantic.chat("Построй столбчатую диаграмму выручки")chart_response.save("artifacts/sales_revenue_chart.png")

### 11.6 Легаси-совместимость: `SmartDataframe` и `SmartDatalake`Чтобы покрыть устаревшие, но всё ещё задокументированные функции (`smart_df.chat()`, `lake.chat()`, `clarification_questions`, `explain`, `follow_up`, `rephrase_query`), подключаем `PostgreSQLConnector`, создаём обёртки и вызываем их.

In [ ]:
from pandasai import SmartDataframe, SmartDatalakefrom pandasai.connectors import PostgreSQLConnectorpostgres_connector = PostgreSQLConnector(config={    "host": "analytics-db",    "port": 5432,    "user": "demo",    "password": "secret",    "database": "revops",})legacy_source = postgres_connector.execute_query("SELECT * FROM sales LIMIT 1000")smart_df = SmartDataframe(legacy_source, config={"llm": primary_llm})legacy_answer = smart_df.chat("Покажи продажи по регионам")lake = SmartDatalake([legacy_source, payroll_raw], config={"llm": primary_llm})lake_answer = lake.chat("Соедини продажи и бонусы")

In [ ]:
def clarification_questions(smart_object, query: str):    return smart_object.chat(f"Какие уточнения нужны для: {query}?")def explain(smart_object, query: str):    return smart_object.chat(f"Объясни шаги расчёта для: {query}")def follow_up(smart_object, query: str):    return smart_object.chat(f"Следующий вопрос: {query}")def rephrase_query(smart_object, query: str):    return smart_object.chat(f"Переформулируй запрос: {query}")clarification_questions(smart_df, "Как рассчитывается маржа?")explain(smart_df, "Почему метрика выросла")follow_up(smart_df, "А как менялся тренд")rephrase_query(smart_df, "Сравни EU и US")clarification_questions(lake, "Дай больше деталей о бонусах")sandbox.stop()